###  **Kannada Text To Audio**

1. **Library Installation**: Installs `parler-tts` for text-to-speech synthesis.

2. **Dependencies**: Uses Hugging Face's `ParlerTTSForConditionalGeneration`, `AutoTokenizer`, and `torch`.

3. **Device Setup**: Determines if GPU (`cuda`) or CPU is used for processing.

4. **Model and Tokenizers**: Loads the `ai4bharat/indic-parler-tts` model and tokenizers for the Kannada text prompt and voice description.

5. **Inputs**:
   - **Prompt**: Kannada text for TTS.
   - **Description**: Defines desired voice characteristics (e.g., monotone, fast).

6. **Audio Generation**: Combines the voice description and text prompt to generate audio using the model.

7. **Save Output**: Writes the generated audio as a `.wav` file (`indic_tts_out.wav`).

8. **Playback**: Plays the synthesized audio in a Jupyter Notebook.


In [20]:
!pip install -q git+https://github.com/huggingface/parler-tts.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done


In [21]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

In [22]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [23]:
model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)

  "_name_or_path": "google/flan-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.46.3",
  "use_cache": true,
  "vocab_size": 32128
}

  "_name_or_path": "ylacombe/dac_44khz",
  "architectures": [
    "DacModel"
  ],
  "codebook_dim": 8,
  "codebook_loss_weight": 1.0,
  "codebook_size": 1024,
  "commitment_loss_weight": 0.25,
  "decoder_hidden_si

In [24]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

In [32]:
prompt = """ಆರು ಹಿತವರು ನಿನಗೆ ಮೂರು ಮಂದಿಗಳೊಳಗೆ
ನಾರಿಯೋ ಧಾರುಣಿಯೋ ಬಲು ಧನದ ಸಿರಿಯೋ"""

In [34]:
description = "Anu's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise. It is slow and clear."

In [35]:
description_input_ids = description_tokenizer(description, return_tensors="pt").to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").to(device)

In [36]:
generation = model.generate(input_ids=description_input_ids.input_ids, attention_mask=description_input_ids.attention_mask, prompt_input_ids=prompt_input_ids.input_ids, prompt_attention_mask=prompt_input_ids.attention_mask)

In [37]:
audio_arr = generation.cpu().numpy().squeeze()

In [38]:
sf.write("indic_tts_out.wav", audio_arr, model.config.sampling_rate)

In [39]:
import IPython
IPython.display.Audio("indic_tts_out.wav")